In [1]:
# Import lib
# ===========================================================
import csv
import pandas as pd
import numpy as np
import random
import time
import collections
import math
import sys
from tqdm import tqdm
from time import sleep

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from datascience import *
from scipy import stats

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
# Initialize useful data
# ===========================================================
# with open('clinvar_conflicting_clean.csv', 'r') as f:
#     reader = csv.reader(f)
#     temp_rows = list(reader)
df = pd.read_csv('clinvar_conflicting_clean.csv', low_memory=False)
columns_to_change = ['ORIGIN', 'EXON', 'INTRON', 'STRAND', 'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'BLOSUM62']
df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']] = df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']].fillna(value="null")

In [3]:
# map categorical data to numerical data
# ===========================================================
def uniq_val(column):
    input_domain = set([column[i][0] for i in range(len(column))])
    return input_domain

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

def map_categ2numer():
    for attribute in df.columns.values:
        
        if attribute == 'CLASS': continue
        # if it's numerical data, normalize it in position
        if is_numeric(df[[attribute]].values[0]):
            # extract the max of the abs(data), take as magnitude
            mag = max(max(df[[attribute]].values), abs(min(df[[attribute]].values)))
            df[[attribute]] = df[[attribute]].div(mag)
        else:  # if it's categorical data, map it according to its index and normalize it
            values_of_this_attrib = uniq_val(df[[attribute]].values)
            length = len(values_of_this_attrib)
            i = 0
            for value in values_of_this_attrib:
    #             print(values_of_this_attrib[i])
                df[[attribute]] = df[[attribute]].replace(value, i / length)
                i += 1

map_categ2numer()
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
0,0.875,0.293401,0.578522,0.799127,0.000000,0.000000,0.000000,0.239983,0.194276,0.527505,...,0.806394,0.0625,0.666667,0.8,0.8,0.535503,0.001056,0.650358,0.999949,0
1,0.875,0.293908,0.578522,0.661572,0.000000,0.515074,0.379013,0.552848,0.898380,0.452553,...,0.588023,0.0625,0.666667,0.8,0.8,0.535503,0.931644,0.090793,0.000000,0
2,0.875,0.294209,0.578522,0.799127,0.000000,0.822409,0.771921,0.239983,0.194276,0.759112,...,0.366952,0.0625,0.666667,0.8,0.8,0.535503,0.259697,0.077541,0.000025,1
3,0.875,0.312446,0.815242,0.661572,0.865939,0.701065,0.511260,0.239983,0.194276,0.467724,...,0.766772,0.0625,0.666667,0.8,0.8,0.535503,0.884697,0.259080,0.000000,0
4,0.875,0.469365,0.815242,0.661572,0.000000,0.084446,0.643987,0.552848,0.898380,0.519298,...,0.218370,0.0625,0.666667,0.8,0.8,0.535503,0.261041,0.072687,0.000000,1


In [6]:
df.to_csv('clinvar_conflicting_mapped.csv', index=False)